In [15]:
# Human Motor Control: Minimum Jerk Trajectory Analysis
# 5 Separate Figures — Noisy data appears ONLY in Figure 5
# Mehdi Delrobaei - Oct. 2025

%matplotlib qt

import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# === Global style for classroom visibility ===
plt.rcParams.update({
    'font.size': 16,
    'axes.titlesize': 18,
    'axes.labelsize': 16,
    'legend.fontsize': 14,
    'xtick.labelsize': 14,
    'ytick.labelsize': 14,
    'figure.dpi': 120,
    'lines.linewidth': 4,
    'lines.markersize': 10
})

# =============================================================================
# 1. DEFINE TRAJECTORY MODELS (NOISE-FREE)
# =============================================================================
def min_jerk_trajectory(t, t0, tf, x0, xf):
    T = tf - t0
    if T <= 0:
        return np.full_like(t, x0)
    tau = (t - t0) / T
    return x0 + (xf - x0) * (10 * tau**3 - 15 * tau**4 + 6 * tau**5)

def min_accel_trajectory(t, t0, tf, x0, xf):
    T = tf - t0
    tau = (t - t0) / T
    return x0 + (xf - x0) * (3 * tau**2 - 2 * tau**3)

def compute_derivatives(x, t):
    dt = t[1] - t[0]
    v = np.gradient(x, dt)
    a = np.gradient(v, dt)
    j = np.gradient(a, dt)
    return v, a, j

# =============================================================================
# 2. GENERATE CLEAN TRAJECTORIES (for Figures 1–4)
# =============================================================================
t0, tf = 0.0, 1.0
x0, xf = 0.0, 20.0
t = np.linspace(t0, tf, 100)

# Clean, analytical trajectories
x_minjerk = min_jerk_trajectory(t, t0, tf, x0, xf)
x_minacc  = min_accel_trajectory(t, t0, tf, x0, xf)
x_linear  = x0 + (xf - x0) * (t - t0) / (tf - t0)

v_minjerk, a_minjerk, j_minjerk = compute_derivatives(x_minjerk, t)
v_minacc,  a_minacc,  j_minacc  = compute_derivatives(x_minacc, t)
v_linear,  a_linear,  j_linear  = compute_derivatives(x_linear, t)

# =============================================================================
# FIGURE 1: Position (Clean Models Only)
# =============================================================================
plt.figure(1, figsize=(10, 6))
plt.plot(t, x_minjerk, 'b-', label='Minimum Jerk')
plt.plot(t, x_minacc,  'r--', label='Minimum Acceleration')
plt.plot(t, x_linear, 'k:', label='Linear')
plt.xlabel('Time (s)')
plt.ylabel('Position (cm)')
plt.title('Figure 1: Position vs. Time (Ideal Models)')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

# =============================================================================
# FIGURE 2: Velocity (Clean Models Only)
# =============================================================================
plt.figure(2, figsize=(10, 6))
plt.plot(t, v_minjerk, 'b-', label='Minimum Jerk')
plt.plot(t, v_minacc,  'r--', label='Minimum Acceleration')
plt.plot(t, v_linear, 'k:', label='Linear')
plt.xlabel('Time (s)')
plt.ylabel('Velocity (cm/s)')
plt.title('Figure 2: Velocity vs. Time (Ideal Models)')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

# =============================================================================
# FIGURE 3: Acceleration (Clean Models Only)
# =============================================================================
plt.figure(3, figsize=(10, 6))
plt.plot(t, a_minjerk, 'b-', label='Minimum Jerk')
plt.plot(t, a_minacc,  'r--', label='Minimum Acceleration')
plt.plot(t, a_linear, 'k:', label='Linear')
plt.xlabel('Time (s)')
plt.ylabel('Acceleration (cm/s²)')
plt.title('Figure 3: Acceleration vs. Time (Ideal Models)')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

# =============================================================================
# FIGURE 4: Jerk (Clean Models Only) ← Critical Comparison
# =============================================================================
plt.figure(4, figsize=(10, 6))
plt.plot(t, j_minjerk, 'b-', label='Minimum Jerk')
plt.plot(t, j_minacc,  'r--', label='Minimum Acceleration')
plt.plot(t, j_linear, 'k:', label='Linear')
plt.xlabel('Time (s)')
plt.ylabel('Jerk (cm/s³)')
plt.title('Figure 4: Jerk vs. Time (Ideal Models)\nMin Jerk = zero at boundaries')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

# =============================================================================
# 5. NOW GENERATE NOISY HUMAN-LIKE DATA (ONLY FOR FIGURE 5)
# =============================================================================
print("Generating noisy human-like data for model fitting...")

np.random.seed(42)  # For reproducibility
t_human = np.linspace(0.05, 1.05, 80)  # Slight timing variability
x_human_clean = min_jerk_trajectory(t_human, 0.0, 1.0, 0.0, 20.0)
x_human = x_human_clean + np.random.normal(0, 0.3, len(t_human))  # Add noise

# Fit minimum jerk model to noisy data
def fit_func(t, t0, tf, x0, xf):
    return min_jerk_trajectory(t, t0, tf, x0, xf)

p0 = [0.0, 1.0, 0.0, 20.0]
bounds = ([-0.5, 0.5, -5, 15], [0.5, 1.5, 5, 25])
popt, _ = curve_fit(fit_func, t_human, x_human, p0=p0, bounds=bounds)

x_pred = fit_func(t_human, *popt)
ss_res = np.sum((x_human - x_pred) ** 2)
ss_tot = np.sum((x_human - np.mean(x_human)) ** 2)
r_squared = 1 - (ss_res / ss_tot)

# =============================================================================
# FIGURE 5: Model Fit to Noisy Human-like Data
# =============================================================================
plt.figure(5, figsize=(10, 6))
plt.plot(t_human, x_human, 'ko', markerfacecolor='black', label='Noisy Human Data')
plt.plot(t_human, x_pred, 'r-', linewidth=4, label=f'Min-Jerk Fit (R² = {r_squared:.3f})')
plt.xlabel('Time (s)')
plt.ylabel('Position (cm)')
plt.title('Figure 5: Fitting Minimum Jerk Model to Noisy Human-like Data')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

# =============================================================================
# Print fitting results
# =============================================================================
print("\n" + "="*60)
print("FITTING RESULTS (Figure 5):")
print(f"  Start time (t₀) = {popt[0]:.3f} s")
print(f"  End time (t_f)   = {popt[1]:.3f} s")
print(f"  Start pos (x₀)   = {popt[2]:.3f} cm")
print(f"  End pos (x_f)    = {popt[3]:.3f} cm")
print(f"  R²               = {r_squared:.4f}")
print("="*60)

Generating noisy human-like data for model fitting...

FITTING RESULTS (Figure 5):
  Start time (t₀) = 0.014 s
  End time (t_f)   = 0.995 s
  Start pos (x₀)   = 0.125 cm
  End pos (x_f)    = 19.992 cm
  R²               = 0.9986
